<a href="https://colab.research.google.com/github/dchung1209/Webtoon-Data-Analysis/blob/main/Naver_Webtoon_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [23]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests

# Scraping Data

In [24]:
# Final Update 25/04/2023

# Query the titles available in on-going webtoons.
def title_query():
  from bs4 import BeautifulSoup
  target = "https://www.webtoons.com/en/dailySchedule"
  r = requests.get(target)
  raw = BeautifulSoup(r.content, 'html.parser')
  title = [p.text for p in raw.find_all('p', {'class': 'subj'})]
  return title

# Query the authors available in on-going webtoons.
def author_query():
  from bs4 import BeautifulSoup
  target = "https://www.webtoons.com/en/dailySchedule"
  r = requests.get(target)
  raw = BeautifulSoup(r.content, 'html.parser')
  author = [p.text for p in raw.find_all('p', {'class': 'author'})]
  return author

# Query the genres available in on-going webtoons.
def genre_query():
  from bs4 import BeautifulSoup
  target = "https://www.webtoons.com/en/dailySchedule"
  r = requests.get(target)
  raw = BeautifulSoup(r.content, 'html.parser')
  genre = [p.text for p in raw.find_all('p', {'class': 'genre'})]
  return genre

def likes_query():
  from bs4 import BeautifulSoup
  target = "https://www.webtoons.com/en/dailySchedule"
  r = requests.get(target)
  raw = BeautifulSoup(r.content, 'html.parser')
  likes = [p.text for p in raw.find_all('em', {'class': 'grade_num'})]
  return likes

def days_query():
  from bs4 import BeautifulSoup
  target = "https://www.webtoons.com/en/dailySchedule"
  r = requests.get(target)
  raw = BeautifulSoup(r.content, 'html.parser')
  days = [p.text for p in raw.find_all('em', {'class': 'grade_num'})]
  return days

In [25]:
titles = title_query()
authors = author_query()
genres = genre_query()
likes = likes_query()

if len(titles) == len(authors) & len(genres) == len(authors):
  print("SUCCESS")
else:
  print("CHECK AGAIN")

SUCCESS


In [26]:
dict = {'title': titles, 'author': authors, 'genre': genres, 'like': likes}
df = pd.DataFrame(dict) 

# Cleaning Data

In [27]:
df.shape

(961, 4)

In [28]:
df.head()

,title,author,genre,like
0,Marry My Husband,sungsojak / LICO,Romance,4.7M
1,SubZero,Junepurrr,Romance,23.6M
2,I’m the Queen in This Life,Themis / Omin,Fantasy,1.8M
3,Cursed Princess Club,LambCat,Comedy,20.8M
4,Tricked into Becoming the Heroine's Stepmother,HARIHEEN / MOKGAMGI,Fantasy,5.4M


In [29]:
# Remove Duplicates

df = df.drop_duplicates()
print("removing duplicates:", df.shape)

removing duplicates: (936, 4)


In [30]:
# Unit Conversion

def convert(s):
  import re
  
  if "M" in s:
    return 1000000 * float(re.sub("[^0-9.\-]","", s))
  elif "," in s:
    return float(re.sub("[^0-9.\-]","", s))

if isinstance(df['like'][0], str):
  df['like'] = df['like'].apply(lambda s: convert(s))

# Download

In [22]:
# to_csv
from datetime import datetime
from google.colab import files

df.to_csv(f"naver_webtoon_{datetime.today().strftime('%Y%m%d')}.csv")
files.download(f"naver_webtoon_{datetime.today().strftime('%Y%m%d')}.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>